In [22]:
import openstudio
import networkx as nx
import matplotlib.pyplot as plt
import pydot
import os

from rdflib import Namespace

import tasty.constants as tc
import tasty.graphs as tg
import tasty.entities as te

In [23]:
BUILDING = Namespace('test:medium_office/')
h_ont = tg.load_ontology(tc.HAYSTACK, tc.V3_9_10)
b_ont = tg.load_ontology(tc.BRICK, tc.V1_2)

# Specify the schema version (tc.V9_9_10, etc.) to use
hp = te.HaystackPointDefs(tc.V3_9_10)
he = te.HaystackEquipDefs(tc.V3_9_10)
hrefs = te.HaystackRefDefs(tc.V3_9_10)

bp = te.BrickPointDefs(tc.V1_2)
be = te.BrickEquipmentDefs(tc.V1_2)
bz = te.BrickZoneDefs(tc.V1_2)
bl = te.BrickLocationDefs(tc.V1_2)
brefs = te.BrickRefDefs(tc.V1_2)

# Bind all of the first class types as attributes
hp.bind()
he.bind()
hrefs.bind()

bp.bind()
be.bind()
bz.bind()
bl.bind()
brefs.bind()

# Simple wrapper around all of the shapes
shrap = te.ShapesWrapper(tc.HAYSTACK, tc.V3_9_10)

shrap.bind()
shrap.bind_composite()

def print_graph(g):
    print(g.serialize(format='turtle').decode('utf-8'))

In [16]:
G = nx.DiGraph()
model = openstudio.model.Model().load(openstudio.path(os.path.join(os.path.abspath(''),"data/in.osm"))).get()
connections = model.getObjectsByType(openstudio.IddObjectType("OS:Connection"))
for connection in connections:
    source = connection.getField(2)
    target = connection.getField(4)
    if source.is_initialized() and target.is_initialized():
        sourceObject = openstudio.model.getModelObject(model, openstudio.toUUID(source.get())).get()
        targetObject = openstudio.model.getModelObject(model, openstudio.toUUID(target.get())).get()
        G.add_node(sourceObject.name().get(), object=sourceObject)
        G.add_node(targetObject.name().get(), object=targetObject)
        G.add_edge(sourceObject.name().get(), targetObject.name().get())

zones = model.getObjectsByType(openstudio.IddObjectType("OS:ThermalZone"))
for zone in zones:
    inlet_port = sourceObject = openstudio.model.getModelObject(model, openstudio.toUUID(zone.getField(9).get())).get()
    exhaust_port = sourceObject = openstudio.model.getModelObject(model, openstudio.toUUID(zone.getField(10).get())).get()
    return_port = sourceObject = openstudio.model.getModelObject(model, openstudio.toUUID(zone.getField(12).get())).get()
    G.add_node(inlet_port.name().get(), object=inlet_port)
    G.add_node(exhaust_port.name().get(), object=exhaust_port)
    G.add_node(return_port.name().get(), object=return_port)
    G.add_node(zone.name().get(), object=zone)
    G.add_edge(inlet_port.name().get(), zone.name().get())
    G.add_edge(zone.name().get(), exhaust_port.name().get())
    G.add_edge(zone.name().get(), return_port.name().get())

In [17]:
def get_downstream_subgraph(context, node, stop_at_types=None, stop_at_nodes=None):
    if type(node) != str:
        node = node.name().get()
    nodes = [node]
    while True:
        additions = False
        for node in nodes:
            for node in context.neighbors(node):
                if stop_at_types != None:
                    if get_type(context, node) in stop_at_types:
                        continue
                if stop_at_nodes != None:
                    if node in stop_at_nodes:
                        continue
                if not node in nodes:
                    nodes.append(node)
                    additions = True
        if not additions:
            return context.subgraph(nodes)

def get_upstream_subgraph(context, node, stop_at_types=None, stop_at_nodes=None):
    if type(node) != str:
        node = node.name().get()
    nodes = [node]
    while True:
        additions = False
        for node in nodes:
            for node in context.predecessors(node):
                if stop_at_types != None:
                    if get_type(context, node) in stop_at_types:
                        continue
                if stop_at_nodes != None:
                    if node in stop_at_nodes:
                        continue
                if not node in nodes:
                    nodes.append(node)
                    additions = True
        if not additions:
            return context.subgraph(nodes)

def get_type(context, node):
    object = get_object_from_node(context, node)
    if type(object) == openstudio.openstudioutilitiesidf.WorkspaceObject:
        return object.iddObject().type()
    if type(object) == openstudio.openstudiomodelcore.ModelObject:
        return object.iddObjectType()

def get_nodes_by_type(context, idd_object_type):
    nodes = []
    for node in context.nodes:
        if get_type(context, node) == idd_object_type:
            nodes.append(node)
    return nodes

def get_object_from_node(context, node):
    return nx.get_node_attributes(context, "object")[node]

def name_to_id(name):
    return name.replace(' ','-')

In [18]:
def add_sensor(model, node_object, system_node_property):
    name = node_object.name().get() + '_' + system_node_property.replace(' ','_')
    output_variable = openstudio.openstudiomodel.OutputVariable(system_node_property, model)
    output_variable.setKeyValue(node_object.name().get())
    output_variable.setReportingFrequency('timestep')
    output_variable.setName(name)

    sensor = openstudio.openstudiomodel.EnergyManagementSystemSensor(model, output_variable)
    sensor.setKeyName(str(node_object.handle()))
    sensor.setName(f"EMS_{name}")
    return output_variable

def add_sensor_to_graph(model, node_object, system_node_property, tasty_parent_object, tasty_object, tasty_namespace, tasty_relationship):
    sensor_name = name_to_id(add_sensor(model, node_object, system_node_property).name().get())
    tasty_object.set_namespace(BUILDING)
    tasty_object.set_id(sensor_name)
    tasty_object.add_relationship(tasty_relationship, tasty_parent_object)
    tasty_object.sync()
    return tasty_object

class MetaNode:
    def __init__(self, *nodes):
        self.nodes = {}
        for node in nodes:
            init_node = None
            if type(node) == te.EntityType:
                init_node = node.deep_copy()
            elif type(node) == te.SimpleShape or type(node) == te.CompositeShape:
                init_node = node.cast_to_entity()
            self.nodes[init_node._type_uri.split('#')[0]] = init_node
    
    def set_namespace(self, namespace):
        for node in self.nodes.values():
            node.set_namespace(namespace)

    def set_id(self, id):
        for node in self.nodes.values():
            node.set_id(id)

    def sync(self):
        for node in self.nodes.values():
            node.sync()
    
    def bind_to_graph(self, graph):
        for namespace in graph.namespaces():
            uri = namespace[1].split('#')[0]
            if uri in self.nodes:
                self.nodes[uri].bind_to_graph(graph)

    def add_tags(self, tags, ontology):
        for namespace in ontology.namespaces():
            uri = namespace[1].split('#')[0]
            if uri in self.nodes:
                self.nodes[uri].add_tags(tags, ontology)

    def add_relationship(self, relationship, node):
        if type(relationship) == te.RefType:
            uri = relationship._type_uri.split('#')[0]
            if uri in self.nodes:
                self.nodes[uri].add_relationship(relationship, node.of_URI(uri))
        elif type(relationship) == MetaRef:
            for uri, ref in relationship.all_refs().items():
                if uri in self.nodes:
                    self.nodes[uri].add_relationship(ref, node.of_URI(uri))

    def of_URI(self, uri):
        return self.nodes[uri]


class MetaRef:
    def __init__(self, *refs):
        self.refs = {}
        for ref in refs:
            self.refs[ref._type_uri.split('#')[0]] = ref

    def all_refs(self):
        return self.refs


In [24]:
hg = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
bg = tg.get_versioned_graph(tc.BRICK, tc.V1_2)
hg.bind('medium_office', BUILDING)
bg.bind('medium_office', BUILDING)

site_object = openstudio.openstudiomodel.toBuilding(model.getObjectsByType(openstudio.IddObjectType('OS:Building'))[0]).get()
site = MetaNode(shrap.SiteShape, bl.Building)
site.set_id(name_to_id(site_object.name().get()))
site.set_namespace(BUILDING)
site.bind_to_graph(hg)
site.bind_to_graph(bg)

site_ref = MetaRef(hrefs.siteRef, brefs.hasLocation)


for node in get_nodes_by_type(G, openstudio.IddObjectType('OS:AirloopHVAC')):
    loop_object = openstudio.openstudiomodelhvac.toAirLoopHVAC(get_object_from_node(G, node)).get()

    ahu = MetaNode(he.ahu, be.AHU)
    ahu.set_namespace(BUILDING)
    ahu.set_id(name_to_id(node))
    ahu.add_relationship(site_ref, site)
    ahu.sync()

    outdoor_air_node = loop_object.outdoorAirNode().get()
    mixed_air_node = loop_object.mixedAirNode().get()
    supply_outlet_node = loop_object.supplyOutletNode()
    supply_inlet_node = loop_object.supplyInletNode()
    relief_air_node = loop_object.reliefAirNode().get()

    supply_context = get_downstream_subgraph(G, supply_inlet_node, stop_at_nodes=[node])
    demand_context = get_downstream_subgraph(G, loop_object.demandInletNode(), stop_at_nodes=[node])
    nx.nx_pydot.to_pydot(supply_context).write_png(f"supply_{node}.png")
    nx.nx_pydot.to_pydot(demand_context).write_png(f"demand_{node}.png")

    equip_ref = MetaRef(hrefs.equipRef, brefs.isPartOf)
    point_ref = MetaRef(hrefs.equipRef, brefs.isPointOf)
    air_ref = MetaRef(hrefs.airRef, brefs.isFedBy)
    space_ref = MetaRef(hrefs.spaceRef, brefs.hasLocation)

    add_sensor_to_graph(model, outdoor_air_node, "System Node Temperature", ahu, MetaNode(hp.outside_air_temp_sensor, bp.Outside_Air_Temperature_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, outdoor_air_node, "System Node Relative Humidity", ahu, MetaNode(hp.outside_air_humidity_sensor, bp.Outside_Air_Humidity_Sensor), BUILDING, point_ref)
    oadp = MetaNode(hp.outside_air_temp_sensor, bp.Outside_Air_Dewpoint_Sensor)
    oadp.add_tags(['dewPoint'], h_ont)
    add_sensor_to_graph(model, outdoor_air_node, "System Node Dewpoint Temperature", ahu, oadp, BUILDING, point_ref)
    add_sensor_to_graph(model, outdoor_air_node, "System Node Mass Flow Rate", ahu, MetaNode(hp.outside_air_flow_sensor, bp.Outside_Air_Flow_Sensor), BUILDING, point_ref)

    add_sensor_to_graph(model, mixed_air_node, "System Node Temperature", ahu, MetaNode(hp.air_temp_sensor, bp.Mixed_Air_Temperature_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, mixed_air_node, "System Node Relative Humidity", ahu, MetaNode(hp.air_humidity_sensor, bp.Mixed_Air_Humidity_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, mixed_air_node, "System Node Mass Flow Rate", ahu, MetaNode(hp.air_flow_sensor, bp.Air_Flow_Sensor), BUILDING, point_ref)

    add_sensor_to_graph(model, supply_outlet_node, "System Node Temperature", ahu, MetaNode(hp.discharge_air_temp_sensor, bp.Discharge_Air_Temperature_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, supply_outlet_node, "System Node Relative Humidity", ahu, MetaNode(hp.discharge_air_humidity_sensor, bp.Discharge_Air_Humidity_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, supply_outlet_node, "System Node Mass Flow Rate", ahu, MetaNode(hp.discharge_air_flow_sensor, bp.Discharge_Air_Flow_Sensor), BUILDING, point_ref)

    add_sensor_to_graph(model, supply_inlet_node, "System Node Temperature", ahu, MetaNode(hp.return_air_temp_sensor, bp.Return_Air_Temperature_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, supply_inlet_node, "System Node Relative Humidity", ahu, MetaNode(hp.return_air_humidity_sensor, bp.Return_Air_Humidity_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, supply_inlet_node, "System Node Mass Flow Rate", ahu, MetaNode(hp.return_air_flow_sensor, bp.Return_Air_Flow_Sensor), BUILDING, point_ref)

    add_sensor_to_graph(model, relief_air_node, "System Node Temperature", ahu, MetaNode(hp.exhaust_air_temp_sensor, bp.Exhaust_Air_Temperature_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, relief_air_node, "System Node Relative Humidity", ahu, MetaNode(hp.exhaust_air_humidity_sensor, bp.Exhaust_Air_Humidity_Sensor), BUILDING, point_ref)
    add_sensor_to_graph(model, relief_air_node, "System Node Mass Flow Rate", ahu, MetaNode(hp.exhaust_air_flow_sensor, bp.Exhaust_Air_Flow_Sensor), BUILDING, point_ref)

    for node in get_nodes_by_type(supply_context, openstudio.IddObjectType("OS:Coil:Heating:Gas")):
        coil = MetaNode(shrap.GasHeatingCoilShape, be.Heating_Coil)
        coil.set_id(name_to_id(node))
        coil.set_namespace(BUILDING)
        coil.add_relationship(equip_ref, ahu)
        coil.sync()

    for node in get_nodes_by_type(supply_context, openstudio.IddObjectType("OS:Coil:Cooling:DX:TwoSpeed")):
        coil = MetaNode(shrap.DXCoolingCoilShape, be.Cooling_Coil)
        coil.set_id(name_to_id(node))
        coil.set_namespace(BUILDING)
        coil.add_relationship(equip_ref, ahu)
        coil.sync()

    for node in get_nodes_by_type(supply_context, openstudio.IddObjectType("OS:Fan:VariableVolume")):
        fan = MetaNode(shrap.VAVFanShape, be.Discharge_Fan)
        fan.set_id(name_to_id(node))
        fan.set_namespace(BUILDING)
        fan.add_tags(['discharge'], h_ont)
        fan.add_relationship(equip_ref, ahu)
        fan.sync()

    zones = get_nodes_by_type(demand_context, openstudio.IddObjectType("OS:ThermalZone"))
    multi_zones = len(zones) > 1
    for node in zones:
        zone_context = get_upstream_subgraph(demand_context, node, stop_at_types=[openstudio.IddObjectType("OS:AirLoopHVAC:ZoneSplitter")])
        zone_object = openstudio.openstudiomodelhvac.toThermalZone(get_object_from_node(demand_context, node)).get()
        zone_return_object = zone_object.returnAirModelObject().get()
        nx.nx_pydot.to_pydot(zone_context).write_png(f"{node}.png")
        zone = MetaNode(shrap.HVACZoneShape, bz.HVAC_Zone)
        zone.set_id(name_to_id(node))
        zone.set_namespace(BUILDING)
        zone.add_relationship(hrefs.siteRef, site)

        for node in get_nodes_by_type(zone_context, openstudio.IddObjectType("OS:AirTerminal:SingleDuct:VAV:Reheat")):
            terminal_object = openstudio.openstudiomodelstraightcomponent.toAirTerminalSingleDuctVAVReheat(get_object_from_node(demand_context, node)).get()
            terminal_inlet_object = terminal_object.inletModelObject().get()
            terminal_outlet_object = terminal_object.outletModelObject().get()

            terminal = MetaNode(he.vav, be.VAV)
            terminal.set_id(name_to_id(node))
            terminal.set_namespace(BUILDING)
            terminal.add_relationship(air_ref, ahu)
            terminal.add_relationship(site_ref, site)
            terminal.sync()
            zone.add_relationship(air_ref, terminal)

            coil_object = terminal_object.reheatCoil()
            coil = MetaNode(shrap.ElecHeatingCoilShape, be.Heating_Coil)
            coil.set_id(name_to_id(coil_object.name().get()))
            coil.add_tags(['reheats'], h_ont)
            coil.set_namespace(BUILDING)
            coil.add_relationship(equip_ref, terminal)

            hcdat = MetaNode(hp.discharge_air_temp_sensor, bp.Discharge_Air_Temperature_Sensor)
            hcdat.add_tags(['heatingCoil'], h_ont)
            hcdarh = MetaNode(hp.discharge_air_humidity_sensor, bp.Discharge_Air_Humidity_Sensor)
            hcdarh.add_tags(['heatingCoil'], h_ont)
            hceps = MetaNode(shrap.ElecHeatingPowerSensorShape, bp.Electrical_Power_Sensor)
            add_sensor_to_graph(model, terminal_outlet_object, "System Node Temperature", coil, hcdat, BUILDING, point_ref)
            add_sensor_to_graph(model, terminal_outlet_object, "System Node Relative Humidity", coil, hcdarh, BUILDING, point_ref)
            add_sensor_to_graph(model, coil_object, "Heating Coil Electricity Rate", coil, hceps, BUILDING, point_ref)

            coil.sync()
        zone.sync()

        if multi_zones:
            zrat = MetaNode(hp.return_air_temp_sensor, bp.Return_Air_Temperature_Sensor)
            zrat.add_tags(['zone'], h_ont)
            zrarh = MetaNode(hp.return_air_humidity_sensor, bp.Return_Air_Humidity_Sensor)
            zrarh.add_tags(['zone'], h_ont)
            zraf = MetaNode(hp.return_air_flow_sensor, bp.Return_Air_Flow_Sensor)
            zraf.add_tags(['zone'], h_ont)
            add_sensor_to_graph(model, zone_return_object, "System Node Temperature", zone, zrat, BUILDING, point_ref)
            add_sensor_to_graph(model, zone_return_object, "System Node Relative Humidity", zone, zrarh, BUILDING, point_ref)
            add_sensor_to_graph(model, zone_return_object, "System Node Mass Flow Rate", zone, zraf, BUILDING, point_ref)

        zat = MetaNode(hp.air_temp_sensor, bp.Zone_Air_Temperature_Sensor)
        zat.add_tags(['zone'], h_ont)
        add_sensor_to_graph(model, zone_object.zoneAirNode(), "System Node Temperature", zone, zat, BUILDING, point_ref)

        zarh = MetaNode(hp.air_humidity_sensor, bp.Zone_Air_Humidity_Sensor)
        zarh.add_tags(['zone'], h_ont)
        add_sensor_to_graph(model, zone_object.zoneAirNode(), "System Node Relative Humidity", zone, zarh, BUILDING, point_ref)
with open("output_haystack.ttl","w") as out:
    out.write(hg.serialize(format='turtle').decode('utf-8'))

graph
Bound test:medium_office/Perimeter_mid_ZN_2-ZN-Zone-Air-Node_System_Node_Relative_Humidity-1 to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-VAV-Terminal to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-VAV-Terminal to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-Electric-Reheat-Coil to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-Electric-Reheat-Coil to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-VAV-Terminal-Outlet-Air-Node_System_Node_Temperature-1 to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-VAV-Terminal-Outlet-Air-Node_System_Node_Temperature-1 to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-VAV-Terminal-Outlet-Air-Node_System_Node_Relative_Humidity-1 to graph
Bound test:medium_office/Perimeter_mid_ZN_1-ZN-VAV-Terminal-Outlet-Air-Node_System_Node_Relative_Humidity-1 to graph
Bound test:medium_office/Perimet

In [21]:
with open("output_haystack.json","w") as out:
    out.write(tg.graph_to_hayson_string(hg))

with open("output_brick.ttl","w") as out:
    out.write(bg.serialize(format='turtle').decode('utf-8'))


In [8]:
print_graph(bg)

@prefix brick: <https://brickschema.org/schema/Brick#> .

<test:medium_office/-MediumOffice-ASHRAE-169-2013-2A-created:-2021-05-19-12:07:23--0600> a brick:Building .




In [9]:
type(shrap.VAVFanShape)

tasty.entities.SimpleShape

In [45]:
hasattr(openstudio.openstudiomodelstraightcomponent, 'to'+openstudio.IddObjectType('OS:WaterUse:Connections').valueDescription().replace('OS','').replace(':','').replace('_',''))


True

In [50]:
import inspect
for member in inspect.getmembers(openstudio):
    if inspect.ismodule(member[1]):
        print(member)

('_openstudioairflow', <module 'openstudio._openstudioairflow' from '/Users/tshapins/Library/Caches/pypoetry/virtualenvs/openstudio-metadata-utility-BJz8s70h-py3.9/lib/python3.9/site-packages/openstudio/_openstudioairflow.so'>)
('_openstudioenergyplus', <module 'openstudio._openstudioenergyplus' from '/Users/tshapins/Library/Caches/pypoetry/virtualenvs/openstudio-metadata-utility-BJz8s70h-py3.9/lib/python3.9/site-packages/openstudio/_openstudioenergyplus.so'>)
('_openstudiogbxml', <module 'openstudio._openstudiogbxml' from '/Users/tshapins/Library/Caches/pypoetry/virtualenvs/openstudio-metadata-utility-BJz8s70h-py3.9/lib/python3.9/site-packages/openstudio/_openstudiogbxml.so'>)
('_openstudioisomodel', <module 'openstudio._openstudioisomodel' from '/Users/tshapins/Library/Caches/pypoetry/virtualenvs/openstudio-metadata-utility-BJz8s70h-py3.9/lib/python3.9/site-packages/openstudio/_openstudioisomodel.so'>)
('_openstudiomeasure', <module 'openstudio._openstudiomeasure' from '/Users/tshapi

In [12]:
hg

NameError: name 'hg' is not defined